# Dungeons and Dragons Companion Toolkit

This notebook is meant to be a lightweight toolkit for rolling dice for ability checks and damage in the tabletop game Dungeons and Dragons

## Imports

In [ ]:
import random
import json
import re

## Json loads and function definitions

In [ ]:
def import_char_json(filepath):
    with open(filepath) as file:
        char_sheet = json.load(file)
    return char_sheet

In [ ]:
# load the json formatted character sheet. Example is below
with open('roll-initiative/char_sheet.json') as file:
    raw_sheet = json.load(file)

In [ ]:
char_sheet = {'character': {'name': 'Ur-Shanai',
  'race': 'High Elf',
  'class': 'Cleric',
  'subclass': 'Grave Domain',
  'level': 5},
 'abilities': {'strength': {'num': 13, 'mod': 1},
  'dexterity': {'num': 18, 'mod': 4},
  'constitution': {'num': 18, 'mod': 4},
  'intelligence': {'num': 14, 'mod': 2},
  'wisdom': {'num': 17, 'mod': 3},
  'charisma': {'num': 16, 'mod': 3},
  'nomod': {'num': 12, 'mod': 0}},
 'saving_throws': {'strength': 1,
  'dexterity': 4,
  'constitution': 4,
  'intelligence': 2,
  'wisdom': 6,
  'charisma': 6},
 'passive_senses': {'perception': 16,
  'investigation': 12,
  'insight': 16,
  'misc': ['darkvision 60ft']}},

skill_sheet =  {'acrobatics': 'dexterity',
                'animal_handling': 'wisdom',
                'arcana': 'intelligence',
                'athletics': 'strength',
                'deception': 'charisma',
                'history': 'intelligence',
                'insight': 'wisdom',
                'intimidation': 'charisma',
                'investigation': 'intelligence',
                'medicine': 'wisdom',
                'nature': 'intelligence',
                'perception': 'wisdom',
                'performance': 'charisma',
                'persuasion': 'charisma',
                'religion': 'intelligence',
                'sleightofhand': 'dexterity',
                'stealth': 'dexterity',
                'survival': 'wisdom',
                'nomod' : 'nomod'}

In [109]:
# base dice rolling function
def dice_roll(x = 20, addl_dice = 0, ability = '', numdice = 1, show_output = False, roll_type = 'ability', crit = False):
    ''' dice_roll(x, modifier, numdice)
        
        x = int n sides of the dice (default d20)
        addl_dice = int n sides of added die, granted from a spell like Guidance (default 0)
        modifier = int value of modifier for roll (default 0)
        numdice = int n number of dice (default 1)
        crit = whether or not the hit roll was a critical hit (default False)
        

        This function returns the final total for a roll in D&D
        Prints each dice roll, base sum, modifier, and the total roll value
        This function handles critical hits by duplicating the damage die, then adding the ability modifier.
        
        
        Returns dice, additional_dice, modifier
    '''
    count = 1
    dice = 0
    add_dice = 0
    output = ''
    try:
        modifier = char_sheet['abilities'][ability]['mod']
    except Exception:
        char_lookup = skill_sheet[ability]
        modifier = char_sheet['abilities'][char_lookup]['mod']
    for i in range(numdice):
        count += 1
        roll = random.randint(1,x)
        output += f'<|>  d{x} #{count - 1} Dice Roll :  {roll}  <|>\n'
        dice += roll
    if crit == True:
        dice *= 2
    if addl_dice != 0:
            add_dice += random.randint(1,addl_dice)
            output += f'<|>  d{addl_dice} Bonus Roll   :  {add_dice}  <|>\n'
    output += f'''\n\n Dice Total          :  {dice + add_dice}\n Modifier            :  {modifier:{"+" if modifier else ""}}\n-----------------------------\n Total Roll          :  {dice + modifier + add_dice}'''
    if show_output:
        print(output)
    return dice, add_dice, modifier

## Example dice rolls

####  6d10 survival check
purposely made it an absurd roll to test

In [106]:
dice_type = 10
addl_dice = 0
modifier = 'survival'
number_of_dice = 6
crit = False
dice, add_dice, modifier = dice_roll(dice_type, 
                                             addl_dice, 
                                             modifier, 
                                             number_of_dice, 
                                             show_output= True) 
print(output)

<|>  d10 #1 Dice Roll :  7  <|>
<|>  d10 #2 Dice Roll :  10  <|>
<|>  d10 #3 Dice Roll :  4  <|>
<|>  d10 #4 Dice Roll :  1  <|>
<|>  d10 #5 Dice Roll :  10  <|>
<|>  d10 #6 Dice Roll :  4  <|>


 Dice Total          :  36
 Modifier            :  +3
-----------------------------
 Total Roll          :  39
<|>  d20 #1 Dice Roll :  4  <|>


 Dice Total          :  4
 Modifier            :  +4
-----------------------------
 Total Roll          :  8


In [ ]:
def dice_to_roll(str):
    ''' Parses a roll string with modifier and extracts the type of dice, how many, and what modifier'''
    split = tuple(str.split('d', maxsplit = 1))
    number_of_dice = split[0]
    dice_type = split[1].split()[0]
    modifier = split[1].split()[1]
    roll_type = split[1].split()[-1]
    return number_of_dice, dice_type, modifier, roll_type

In [ ]:
def crit_handler(roll_type, roll, dice_type):
    if roll_type == 'hit' and roll == dice_type:
        crit = True
    else:
        crit = False
    return crit
    
    

In [ ]:
def dice_roll2(x = 20, addl_dice = 0, ability = '', numdice = 1, show_output = False):
    ''' dice_roll(x, modifier, numdice)
        
        x = int n sides of the dice (default d20)
        addl_dice = int n sides of added die, granted from a spell like Guidance (default 0)
        modifier = int value of modifier for roll (default 0)
        numdice = int n number of dice (default 1)
        crit = whether or not the hit roll was a critical hit (default False)
        

        This function returns the final total for a roll in D&D
        Prints each dice roll, base sum, modifier, and the total roll value
        This function handles critical hits by duplicating the damage die, then adding the ability modifier.
        
        
        Returns dice, additional_dice, modifier
    '''
    count = 1
    dice = 0
    add_dice = 0
    output = ''
    try:
        modifier = char_sheet['abilities'][ability]['mod']
    except Exception:
        char_lookup = skill_sheet[ability]
        modifier = char_sheet['abilities'][char_lookup]['mod']
    for i in range(numdice):
        count += 1
        roll = random.randint(1,x)
        output += f'<|>  d{x} #{count - 1} Dice Roll :  {roll}  <|>\n'
        dice += roll
    if crit == True:
        dice *= 2
    if addl_dice != 0:
            add_dice += random.randint(1,addl_dice)
            output += f'<|>  d{addl_dice} Bonus Roll   :  {add_dice}  <|>\n'
    output += f'''\n\n Dice Total          :  {dice + add_dice}\n Modifier            :  {modifier:{"+" if modifier else ""}}\n-----------------------------\n Total Roll          :  {dice + modifier + add_dice}'''
    if show_output:
        print(output)
    return dice, add_dice, modifier

In [115]:
# Functional version since I use this regularly in my DnD groupß
dice_roll(x = 20,
          addl_dice = 0,
          ability = 'acrobatics',
          numdice = 2,
          show_output = True,
          crit = False)

<|>  d20 #1 Dice Roll :  8  <|>
<|>  d20 #2 Dice Roll :  10  <|>


 Dice Total          :  18
 Modifier            :  +4
-----------------------------
 Total Roll          :  22


(18, 0, 4)

To Do:

write the main function that uses all the functions. The main dice roller may need to be refactored or split up into more simple funcs to handle whether or not a roll is a critical hit. Probably just rewrite all that junk since I'm restructuring the interface. 

I'd like to package this in a UI, I'll probably create a webapp and run it locally so that users can upload thier charactersheets as jsons and then also interface and conduct dice rolls in the app. 

The recent updates are just compartmentalizing different actions in the functions so that I may later make a "dice roll" class object. Doing that in a clever manner would allow this pet project to be used in other dice games beyond DnD. 

End goal is to have a dice rolling library that I import and can roll dice through the command line. I need to build a function to read CLI args and output the roll. Eventually, the UI needs to be developed to handle different n-sided die, any number of said die, any modifying die, and also handle crit attack rolls correctly. 